In [46]:
import pandas as pd
import numpy as np

In [22]:
#数据预处理
def Preprocessing(book_location,ic_location):#参数：book_location:交易风控台所有字段数据的所在磁盘位置。ic_location：ic期货数据   
    data = pd.read_excel(book_location)
    contracts = data[['position_id','market_value','delta_cash','valuation_date']].copy()
    contracts['valuation_date'] = contracts['valuation_date'].apply(lambda x:str(x)[:10])   #日期时间戳格式全部改为字符串格式，方便后面操作
        
    #ic合约数据
    ic_futures = pd.read_excel(ic_location)
    ic_futures['日期'] = ic_futures['日期'].apply(lambda x: str(x)[:10])   #日期时间戳格式全部改为字符串格式，方便后面操作
    ic_futures.rename(columns={'日期':'valuation_date'},inplace=True)
    ic_futures = ic_futures.set_index('valuation_date')
    
    return {'Contracts':contracts,'Ic_futures':ic_futures}

In [30]:
#单个id每日PnL
def Id_PnL(position_id):   #id: position_id字段,ic_futures期货数据
    #产品端PnL
    id_contracts = contracts[contracts['position_id'] == position_id]
    product_pnl = (id_contracts['market_value'].shift(-1) - id_contracts['market_value'])[:-1].to_frame()  
    product_pnl.rename(columns={'market_value':'product_pnl'},inplace=True)
    date_col = id_contracts['valuation_date'].shift(-1)[:-1]   #时间列
    date_col.name = 'date'
    product_pnl.index=date_col  #索引改为日期
    
    
    #对冲端delta cash
    delta_df = id_contracts[['delta_cash','valuation_date']]
    delta_df = delta_df.set_index('valuation_date')
    delta_df = delta_df.join(ic_futures)   #连接上对应日期的IC期货主力合约收盘价
    
    #对冲端盈亏PnL_df
    hedging_df = delta_df.copy()
    
    
    hedging_df['单日对冲手数'] = -round((hedging_df['delta_cash']/(hedging_df['收盘价（主力合约）']*200)))
    hedging_df['ic期货仓位'] = hedging_df['单日对冲手数'].cumsum()
    hedging_df['单日对冲成本'] = hedging_df['单日对冲手数']*hedging_df['收盘价（主力合约）']*200   #对冲成本：正，表示买入；负，表示卖出
    hedging_df['累计成本']=hedging_df['单日对冲成本'].cumsum()
    hedging_df['已实现盈亏'] = -hedging_df['单日对冲成本']
    hedging_df['一手成本'] = hedging_df['累计成本']/hedging_df['ic期货仓位']
    # hedging_df['浮动盈亏'] = hedging_df['ic期货仓位']*(200*hedging_df['收盘价（主力合约）'] - hedging_df['一手成本'])
    hedging_df['浮动盈亏'] = 200*hedging_df['收盘价（主力合约）']*hedging_df['ic期货仓位'] - hedging_df['累计成本']
    hedging_df.index.name = 'date'
    
    #总盈亏 = 产品端盈亏+对冲端盈亏
    pnl_df = hedging_df[['已实现盈亏','浮动盈亏']].join(product_pnl)
    pnl_df.rename(columns={'已实现盈亏': '已实现盈亏'},inplace=True)
    pnl_df.fillna(0,inplace=True)
    pnl_df['总pnl'] = pnl_df['已实现盈亏']+pnl_df['浮动盈亏']+pnl_df['product_pnl']
    pnl_df['position_id'] = position_id
    
    return pnl_df

In [31]:
def All_PnL(book_location, ic_location):
    data = Preprocessing(book_location,ic_location)
    contracts = data['Contracts']
    ic_futures = data['Ic_futures']

    id_l = list(contracts['position_id'].unique())  #合约id去重,得到所有id列表
    pnl_df_l = []   #每个id的pnl_df为列表中一个元素
    for i in range(len(id_l)):
        pnl_df = Id_PnL(id_l[i])
        pnl_df_l.append(pnl_df)
    return pnl_df_l

In [34]:
book_location = 'C:/Users/lenovo/Desktop/资料/暑期资料/场外衍生品/对冲PnL.xlsx'
ic_location = 'C:/Users/lenovo/Desktop/资料/暑期资料/量化/历史数据/IC期货.xlsx'
pnl_df_l = All_PnL(book_location, ic_location)  #返回结果为一个列表，列表中每个元素是一个id的时间序列pnl的dataframe，该dataframe中含有position_id,可以根据这个id来筛选